In [3]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import split, col, sum
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import Binarizer, Normalizer, VectorAssembler
from pyspark.ml.linalg import DenseVector

spark = SparkSession.builder.appName('CosmosDb').getOrCreate()

ratingSchema = StructType(( StructField('user', IntegerType(), False),
                            StructField('item', IntegerType(), False),
                            StructField('rating', FloatType(), False) ))

userSchema = StructType(( StructField('user', IntegerType(), False),
                          StructField('gender', IntegerType(), False),
                          StructField('age', IntegerType(), False) ))

ratings = spark.read.csv('ml-1m/ratings_timesorted.dat', ratingSchema, '::')
users = spark.read.csv('ml-1m/users_binarygender.dat', userSchema, '::')

In [4]:
# Read Configuration
readConfig = {
  "Endpoint" : "https://doctorwho.documents.azure.com:443/",
  "Masterkey" : "SPSVkSfA7f6vMgMvnYdzc1MaWb65v4VQNcI2Tp1WfSP2vtgmAwGXEPcxoYra5QBHHyjDGYuHKSkguHIz1vvmWQ==",
  "Database" : "DepartureDelays",
  "preferredRegions" : "Central US;East US2",
  "Collection" : "flights_pcoll",
  "SamplingRatio" : "1.0",
  "schema_samplesize" : "1000",
  "query_pagesize" : "2147483647",
  "query_custom" : "SELECT c.date, c.delay, c.distance, c.origin, c.destination FROM c WHERE c.origin = 'SEA'"
}

# Connect via azure-cosmosdb-spark to create Spark DataFrame
flights = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
flights.count()

Py4JJavaError: An error occurred while calling o57.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.microsoft.azure.cosmosdb.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:674)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:728)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:203)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at 

In [ ]:
# Write configuration
writeConfig = {
 "Endpoint" : "https://doctorwho.documents.azure.com:443/",
 "Masterkey" : "SPSVkSfA7f6vMgMvnYdzc1MaWb65v4VQNcI2Tp1WfSP2vtgmAwGXEPcxoYra5QBHHyjDGYuHKSkguHIz1vvmWQ==",
 "Database" : "DepartureDelays",
 "Collection" : "flights_fromsea",
 "Upsert" : "true"
}

# Write to Cosmos DB from the flights DataFrame
flights.write.format("com.microsoft.azure.cosmosdb.spark").options(**writeConfig).save()

In [ ]:
spark.stop()